In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf

from scripts.pipeline import Pipeline

2024-03-26 23:53:43.234721: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-26 23:53:43.256934: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-26 23:53:43.256954: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-26 23:53:43.257513: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-26 23:53:43.261346: I tensorflow/core/platform/cpu_feature_guar

In [2]:
#use gpu for tf
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

2024-03-26 23:53:44.221237: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-26 23:53:44.224444: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-26 23:53:44.224477: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [3]:
# Load the data
train_data = pd.read_csv('../data/train.csv')
test_data = pd.read_csv('../data/test.csv')

# split train data into predictors and response
X = train_data.drop(columns=['SalePrice'])
y = train_data['SalePrice'] 

#combine the data for preprocessing
data = pd.concat([X, test_data], axis=0)
# Preprocess the data
data = Pipeline(data)

# Split the data back into train and test, processed
X = data[:len(train_data)]
test = data[len(train_data):]
#increase test index by 1
test.index += 1

# Split the data into features and response
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
#going with extra trees regressor
from sklearn.ensemble import ExtraTreesRegressor
model = ExtraTreesRegressor()
model.fit(X_train, y_train)

# Predict the test data
y_pred = model.predict(test)
test_results = pd.DataFrame({'Id': test.index, 'SalePrice': y_pred})
test_results.to_csv('../outputs/ExtraTreesRegressor_test.csv', index=False)

In [5]:
#going with gradient boosting regressor
from sklearn.ensemble import GradientBoostingRegressor
model = GradientBoostingRegressor()
model.fit(X_train, y_train)

# Predict the test data
y_pred = model.predict(test)
test_results = pd.DataFrame({'Id': test.index, 'SalePrice': y_pred})
test_results.to_csv('../outputs/GradientBoostingRegressor_test.csv', index=False)

In [6]:
# build nn model
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error')

2024-03-26 23:53:46.428336: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-26 23:53:46.428383: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-26 23:53:46.428405: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-26 23:53:46.520821: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-26 23:53:46.520866: I external/local_xla/xla/stream_executor

In [7]:
# Train the model, predict the test data
model.fit(X_train, y_train, epochs=250)
y_pred = model.predict(test).flatten()
test_results = pd.DataFrame({'Id': test.index, 'SalePrice': y_pred})
test_results.to_csv('../outputs/NN128x128_test.csv', index=False)

Epoch 1/400


2024-03-26 23:53:47.173341: I external/local_xla/xla/service/service.cc:168] XLA service 0x7ff87d93ab20 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-26 23:53:47.173363: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2024-03-26 23:53:47.177965: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-03-26 23:53:47.191698: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8906
I0000 00:00:1711497227.232594   24071 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


37/37 [==============================] - 1s 3ms/step - loss: 38885658624.0000
Epoch 2/400
37/37 [==============================] - 0s 2ms/step - loss: 38885392384.0000
Epoch 3/400
37/37 [==============================] - 0s 2ms/step - loss: 38885052416.0000
Epoch 4/400
37/37 [==============================] - 0s 2ms/step - loss: 38884474880.0000
Epoch 5/400
37/37 [==============================] - 0s 2ms/step - loss: 38883622912.0000
Epoch 6/400
37/37 [==============================] - 0s 2ms/step - loss: 38882422784.0000
Epoch 7/400
37/37 [==============================] - 0s 2ms/step - loss: 38880727040.0000
Epoch 8/400
37/37 [==============================] - 0s 2ms/step - loss: 38878498816.0000
Epoch 9/400
37/37 [==============================] - 0s 2ms/step - loss: 38875561984.0000
Epoch 10/400
37/37 [==============================] - 0s 2ms/step - loss: 38871777280.0000
Epoch 11/400
37/37 [==============================] - 0s 2ms/step - loss: 38867116032.0000
Epoch 12/400
37/37 [